In [16]:
#| default_exp indexer

In [17]:
#| export
import ffmpeg
import numpy as np
import pandas as pd

from PIL import Image
from shutil import copy
from pathlib import Path

from projetMAB1.storage import Storage
from projetMAB1.model import Model

In [18]:
#| export
def export_video(video_path: Path, start_frame, end_frame):
    in_file = ffmpeg.input(video_path, loglevel='warning', copyts=None)
    out_file = f"../../data/runtime/fragments/{video_path.stem}_{start_frame}_{end_frame}.mp4"
    (
        ffmpeg.concat(in_file.trim(start_frame=start_frame, end_frame=end_frame))
        .filter('scale', 854 , 480)
        .filter('setsar', '1')
        .filter('setpts', expr='PTS-STARTPTS')
        .output(out_file).run()
    )
    return out_file

In [19]:
#|export
def index(video_path: str, threshold = 0.10):

    video_name = Path(video_path).name
    if Path(video_path) != Path("../../data/runtime/videos/" + video_name):
        copy(video_path, "../../data/runtime/videos/" + video_name)

    out, _ = (
        ffmpeg.input(video_path, loglevel="warning")
        .filter('scale', size='film')
        .output('pipe:', format='rawvideo', pix_fmt='rgb24')
        .run(capture_stdout=True)
    )

    film = 352, 240
    width, height = film

    images = (
        np.frombuffer(out, np.uint8)
        .reshape([-1, height, width, 3])
    )

    df = Storage.load()

    empty_df = True if "video_name" not in df.columns else False

    model = Model()
    image_features = model.get_image_features(images)

    slices = [0]
    threshold = 0.15
    for i in range(1, len(images)):
        if 1 - image_features[i-1] @ image_features[i] >= threshold:
            slices.append(i)
    
    slices.append(len(images)+1)

    entries = []
    video_name = Path(video_path).stem
    for i in range(len(slices)-1):
        start = slices[i]
        end = slices[i+1]
        preview_name = f"{video_name}_{start}_{end}.png"
        entries.append((video_name, preview_name, start, end))

        Image.fromarray(images[slices[i]]).save("../../data/runtime/frames/" + preview_name)
        fragment_video = export_video(Path(video_path),start,end)

    new_df = pd.DataFrame(data = entries, columns= ["video_name", "preview_name", "start", "end"])

    if empty_df:
        df = new_df
    else:
        df = pd.concat([df, new_df], ignore_index=True)

    df.to_pickle("../../data/runtime/index.pkl")

In [20]:
df = pd.read_pickle("../../data/runtime/index.pkl")
df = df.drop(df.index)
df.to_pickle("../../data/runtime/index.pkl")

In [21]:
videos = sorted(Path("../../data/runtime/videos/").glob("*.mp4"))
for video in videos:
    index(str(video))

../../data/runtime/videos/unicorn.mp4


../../data/runtime/videos/video0.mp4
../../data/runtime/videos/video1.mp4
../../data/runtime/videos/video2.mp4
../../data/runtime/videos/video3.mp4


In [22]:
pd.read_pickle("../../data/runtime/index.pkl")

,video_name,preview_name,start,end
0,unicorn,unicorn_0_333.png,0,333
1,video0,video0_0_52.png,0,52
2,video0,video0_52_175.png,52,175
3,video0,video0_175_286.png,175,286
4,video0,video0_286_301.png,286,301
5,video1,video1_0_106.png,0,106
6,video1,video1_106_194.png,106,194
7,video1,video1_194_473.png,194,473
8,video2,video2_0_212.png,0,212
9,video2,video2_212_341.png,212,341


In [23]:
#| hide
import nbdev; nbdev.nbdev_export()